In [ ]:
#@title Readme input
# https://adventofcode.com/2023/day/20
readme_input="""broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a"""

In [ ]:
#@title input
_input="""%pj -> sh
%mn -> jp
&hf -> rg, vl, tq, qq, mv, zz
%xl -> hf
%sv -> mn, dl
%kk -> lh
&sj -> kz
%jj -> lq, kk
%mr -> bm, hb
%sx -> lq, fn
%fn -> zr, lq
%pf -> dl, gv
%lr -> jj, lq
%jp -> dl, pj
&hb -> sj, mr, rz, qg, pr
%vg -> zz, hf
%pr -> zq
%hn -> pf
%jg -> tj
%qg -> vk
%dv -> xl, hf
&qq -> kz
%fm -> lr
&ls -> kz
%pd -> hb, xg
%rj -> hb
%fb -> hf, tq
%zz -> np
%bm -> pd, hb
%xn -> lq, fm
%gv -> jg, dl
%dz -> sx
%zs -> dl, nh
%tj -> zs, dl
%mv -> vl
&kz -> rx
%np -> cl, hf
&bg -> kz
%vl -> vg
%xg -> rz, hb
%rz -> pr
%zq -> hb, qg
%lh -> rd
%zr -> lq
%fl -> hb, rj
%xr -> xn, lq
%rd -> dz, lq
%cl -> hf, gj
%nh -> dl
%sh -> hn, dl
%vk -> fx, hb
%gj -> hf, dv
%tq -> mv
&dl -> hn, pj, ls, mn, jg, sv
%fx -> fl, hb
&lq -> bg, kk, dz, xr, lh, fm
%rg -> hf, fb
broadcaster -> xr, mr, rg, sv"""

In [ ]:
from pprint import pprint

LOW = -1
HIGH = 1
OFF = 0
ON = 2

def parser(s):
    flip_flop = {}
    conjunction = {}
    for line in s.splitlines():
        key, values = line.split(' -> ')
        values = values.split(', ')
        if key.startswith("broadcaster"):
            broadcaster = values
        elif key.startswith("%"):
            flip_flop[key[1:]] = values
        elif key.startswith("&"):
            conjunction[key[1:]] = values

    return broadcaster, flip_flop, conjunction


pprint(parser(readme_input))

(['a', 'b', 'c'], {'a': ['b'], 'b': ['c'], 'c': ['inv']}, {'inv': ['a']})


In [ ]:
from collections import deque

def part1(broadcaster, flip_flop, conjunction):
    low = 0
    high = 0

    flip_flop_state = {
        x:OFF for x in flip_flop
    }

    conjunction_inputs = {}
    for k in conjunction:
        inputs = []
        for i, vs in flip_flop.items():
            for v in vs:
                if v==k:  # i is an input of k
                    inputs.append(i)
        conjunction_inputs[k] = {
            x:LOW for x in inputs
        }

    def send_high(_type, node):
        for dest_node in _type[node]:
            stack.append((HIGH, dest_node, node))

    def send_low(_type, node):
        for dest_node in _type[node]:
            stack.append((LOW, dest_node, node))

    for _ in range(1000):
        stack = deque([(LOW, "broadcaster", "~~~BUTTON~~~")])
        while stack:
            signal, node, _from = stack.popleft()
            if signal==LOW:
                low+=1
            else:
                high+=1
            # print(f'{_from} -{"LOW" if signal==LOW else "HIGH"}-> {node}')
            if node == "broadcaster":
                for n in broadcaster:
                    stack.append((signal, n, node))
                continue

            if node in flip_flop and signal==LOW:
                if flip_flop_state[node] == OFF:
                    flip_flop_state[node] = ON
                    send_high(flip_flop, node)
                elif flip_flop_state[node] == ON:
                    flip_flop_state[node] = OFF
                    send_low(flip_flop, node)

            if node in conjunction:
                conjunction_inputs[node][_from] = signal
                if all(x==HIGH for x in conjunction_inputs[node].values()):
                    send_low(conjunction, node)
                else:
                    send_high(conjunction, node)

    return low*high

print(part1(*parser(readme_input)))
print(part1(*parser(_input)))

32000000
944750144


In [ ]:
from itertools import count

def part2(broadcaster, flip_flop, conjunction, wait_for='rx'):
    flip_flop_state = {
        x:OFF for x in flip_flop
    }

    conjunction_inputs = {}
    for k in conjunction:
        inputs = []
        for i, vs in flip_flop.items():
            for v in vs:
                if v==k:  # i is an input of k
                    inputs.append(i)
        conjunction_inputs[k] = {
            x:LOW for x in inputs
        }

    def send_high(_type, node):
        for dest_node in _type[node]:
            stack.append((HIGH, dest_node, node))

    def send_low(_type, node):
        for dest_node in _type[node]:
            stack.append((LOW, dest_node, node))

    start = None
    for step in count(start=1):
        stack = deque([(LOW, "broadcaster", "~~~BUTTON~~~")])
        while stack:
            signal, node, _from = stack.popleft()
            if node==wait_for and signal==LOW:
                return step
            # print(f'{_from} -{"LOW" if signal==LOW else "HIGH"}-> {node}')
            if node == "broadcaster":
                for n in broadcaster:
                    stack.append((signal, n, node))
                continue

            if node in flip_flop and signal==LOW:
                if flip_flop_state[node] == OFF:
                    flip_flop_state[node] = ON
                    send_high(flip_flop, node)
                elif flip_flop_state[node] == ON:
                    flip_flop_state[node] = OFF
                    send_low(flip_flop, node)

            if node in conjunction:
                conjunction_inputs[node][_from] = signal
                if all(x==HIGH for x in conjunction_inputs[node].values()):
                    send_low(conjunction, node)
                else:
                    send_high(conjunction, node)
    # return low*high

# see https://kroki.io/mermaid/png/eNpl1EtuxCAMANC9TzGr7HqFrnqEXmAYCogA5RMhxOkLMTbSdPfk8LMx0fkZzeP7C6J9fHx8PooBH27ZCIdRN7NmVse8EvNI274ye4fmcIBRUCp-D6RDOjjPm87AUfAEx9nBLrpEPE_w-ZbwpMMIKI1HLqoACjPomTQ_R8WbLuoKDleylnRvGXnkYrQjJ4HRsu0zM3dm0syYoWoqxdIsRcTNegKDB4wKLH6-LCRUPUFioZojzcmj2Fwo5bF84yCucDRKrs9i05AtB5XgxRbHXfZ-K0QQuGiUpDmn7UouKg8aT2U1adZM4kKlQS8cXAwGLjxHL6T52VfurhOn5wYBi_9ypDt7oTnPis1VNTRNt9A0Hzl3uoSeOLg4LskZnCOh77tXblcJmS00_N4CaY7MWFnZSTP42u2-qC0Ew3kWpAmkGaz4BlQjzd215ZUWZYUr8ROTjlYiziZdHM3A9nvEuCdiqaDWe3Gkue1IAi00e7w94siW2DJzvt7F0RR59_miEiDy71O-nuX6yTT7PeT_h8aP5z1U6h_x2DWj
print(
    part2(*parser(_input), wait_for='bg') *
    part2(*parser(_input), wait_for='ls') *
    part2(*parser(_input), wait_for='qq') *
    part2(*parser(_input), wait_for='sj')
    )
222718819437131



222718819437131


222718819437131

![y2023day20part2.png](https://kroki.io/mermaid/svg/eNpl1EtuxCAMANC9TzGr7HqFrnqEXmAYCogA5RMhxOkLMTbSdPfk8LMx0fkZzeP7C6J9fHx8PooBH27ZCIdRN7NmVse8EvNI274ye4fmcIBRUCp-D6RDOjjPm87AUfAEx9nBLrpEPE_w-ZbwpMMIKI1HLqoACjPomTQ_R8WbLuoKDleylnRvGXnkYrQjJ4HRsu0zM3dm0syYoWoqxdIsRcTNegKDB4wKLH6-LCRUPUFioZojzcmj2Fwo5bF84yCucDRKrs9i05AtB5XgxRbHXfZ-K0QQuGiUpDmn7UouKg8aT2U1adZM4kKlQS8cXAwGLjxHL6T52VfurhOn5wYBi_9ypDt7oTnPis1VNTRNt9A0Hzl3uoSeOLg4LskZnCOh77tXblcJmS00_N4CaY7MWFnZSTP42u2-qC0Ew3kWpAmkGaz4BlQjzd215ZUWZYUr8ROTjlYiziZdHM3A9nvEuCdiqaDWe3Gkue1IAi00e7w94siW2DJzvt7F0RR59_miEiDy71O-nuX6yTT7PeT_h8aP5z1U6h_x2DWj)